### 8-ball shot

![alt text](image.png)

Conservation of momentum: $m_A {v_A}_i + m_B {v_B}_i = m_A {v_A}_f + m_B {v_B}_f$

Coefficient of restitution: $e = -\frac{{v_B}_f-{v_A}_f}{{v_B}_i-{v_A}_i}$

Conservation of energy: ${{E_K}_A}_i + {{E_K}_B}_i = {{E_K}_A}_f + {{E_K}_B}_f$

Mechanical kinetic energy: $E_{K_\text{mechanic}} = \frac{1}{2}mv^2$

In [37]:
%matplotlib widget
# Dependencies
import matplotlib.animation
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import patches

# Settings
plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 150
w_ = 2.54
h_ = 1.27

In [38]:
class Ball(patches.Circle):
    def __init__(self, xy, ax, vel=[0,0], **kwargs):
        rad = 0.05715  # Fixed radius
        super().__init__(xy, rad, **kwargs)
        ax.add_patch(self)
        self.vel = np.array(vel).astype(np.float64)
        self.acc = np.zeros(2).astype(np.float64)
        
    def get_xy(self):
        return np.array(self.get_center()).astype(np.float64)

    def update(self, dti):
        if self.get_visible():
            pos = self.get_xy()
            if not np.linalg.vector_norm(self.vel) == 0:
                self.acc = - 0.6 * self.vel / np.linalg.vector_norm(self.vel)
            else:
                self.acc = np.zeros(2).astype(np.float64)
            self.vel += self.acc * dti
            if pos[0] + self.radius >= w_ or pos[0] - self.radius <= 0:
                self.vel[0] *= -1
            if pos[1] + self.radius >= h_ or pos[1] - self.radius <= 0:
                self.vel[1] *= -1
            pos += self.vel * dti
            self.set_center(pos)
        

Solving for velocities after collision:

$
\left[ \begin{array}{rll}
    e =& -\frac{{v_b}_f-{v_a}_f}{{v_b}_i-{v_a}_i} & (\text{Coefficient of restitution}) \\
    {v_a}_i+{v_b}_i =& {v_a}_f+{v_b}_f & (\text{Conservation of momentum for } m_a = m_b)
\end{array} \right.
$

$e = 1 \rightarrow
\left[ \begin{array}{rl}
    1 =& -\frac{{v_b}_f-{v_a}_f}{{v_b}_i-{v_a}_i} \\
    {v_a}_i+{v_b}_i =& {v_a}_f+{v_b}_f
\end{array} \right.
$

$
\left[ \begin{array}{rl}
    ({v_a}_i-{v_b}_i) =& {v_b}_f-{v_a}_f \\
    {v_a}_i+{v_b}_i =& {v_a}_f+{v_b}_f
\end{array} \right.
$

$
\left[ \begin{array}{rl}
    {v_b}_f =& {v_a}_f + {v_a}_i - {v_b}_i \\
    {v_b}_f =& {v_a}_i + {v_b}_i - {v_a}_f
\end{array} \right.
$

$\rightarrow {v_a}_f + {v_a}_i - {v_b}_i = {v_a}_i + {v_b}_i - {v_a}_f$

$\therefore {v_a}_f = {v_b}_i$

$\therefore {v_b}_f = {v_a}_i$

Check for a collision: $\|\mathbf{p}_{b_2}-\mathbf{p}_{b_1}\| \leq r_{b_1} + r_{b_2}$
```python
if np.linalg.norm(p_b2 - p_b1) <= (r_b1 + r_b2):
    ...
```

Normal axis vector: $\mathbf{n} = \frac{\mathbf{p}_{b_2}-\mathbf{p}_{b_1}}{\|\mathbf{p}_{b_2}-\mathbf{p}_{b_1}\|}$

```python
n_ax = (p_b2 - p_b1) / np.linalg.norm(p_b2 - p_b1)
```

Velocity in the normal axis: $\mathbf{v}_n = (\mathbf{v} \cdot \mathbf{n})\ \mathbf{n}$

```python
v_n = np.dot(v, n_ax) * n_ax
```

Velocity in the tangential axis: $\mathbf{v}_t = \mathbf{v} - \mathbf{v}_n$

```python
v_t = v - v_n
```

In [39]:
""" YOUR CODE IN THIS BLOCK """

def estimate_cue_vel(b_c, b_8, pholes):
    """ Estimate initial velocity of cue ball """
    pcb = b_c.get_xy()
    p8b = b_8.get_xy()
    rcb = b_c.get_radius()
    r8b = b_8.get_radius()
    
    best_hole = 0
    best_dotp = 0
    for phole in pholes:
        current_dotp = np.dot((phole - p8b) / np.linalg.norm(phole - p8b),
                              (p8b - pcb) / np.linalg.norm(p8b - pcb))
        best_hole = phole if current_dotp > best_dotp else best_hole
        best_dotp = np.max([current_dotp, best_dotp])

    nrm = (p8b - best_hole) / np.linalg.norm(p8b - best_hole)
    npb = p8b + (rcb + r8b)*nrm

    pdiff = (npb - pcb) / np.linalg.norm(npb - pcb)
    vel = 3*pdiff
    return vel

def collision(b_1, b_2):
    """ Check and apply collisions between b_1 and b_2 """
    p_b1 = b_1.get_xy()
    r_b1 = b_1.get_radius()
    v_b1 = b_1.vel
    p_b2 = b_2.get_xy()
    r_b2 = b_2.get_radius()
    v_b2 = b_2.vel

    # Solution
    p_d = p_b2 - p_b1
    dst = np.linalg.norm(p_d)
    if (r_b1 + r_b2) >= dst :
        n_d = p_d / dst
        b_1.set_center(p_b1 + 0.5*(dst - (r_b1+r_b2))*n_d)
        b_2.set_center(p_b2 - 0.5*(dst - (r_b1+r_b2))*n_d)
        vb1n = np.dot(v_b1, n_d) * n_d
        vb2n = np.dot(v_b2, n_d) * n_d
        vb1t = (v_b1 - vb1n)
        vb2t = (v_b2 - vb2n)
        vb1f = vb1t + vb2n
        vb2f = vb2t + vb1n
        v_b1 = vb1f
        v_b2 = vb2f

    b_1.vel = v_b1
    b_2.vel = v_b2


In [ ]:
try:
    ani.event_source.stop()
except (NameError, AttributeError) as e:
    pass

# Plot setup
fig = plt.figure(figsize=(6,3))
fig.set_label("8 ball shot")
ax = fig.subplots()
ax.set_facecolor((0,0.5,0))
ax.set_xlim(0, w_)
ax.set_ylim(0, h_)
ax.set_xticks([])
ax.set_yticks([])

# Holes
ax.vlines(w_/4, 0, h_, 'k', linewidth=1, color=(.1,.1,.1), zorder=0)
p_holes = [[xi, yi] for yi in [0, h_] for xi in [0, w_/2, w_]]
holes = []
for p_hi in p_holes:
    hole = patches.Circle(p_hi, 0.08, color=(0,0,0))
    holes.append(hole)
    ax.add_patch(hole)

# Cue ball and 8 ball
p_cb = np.array([w_/4, h_/2])
p_8b = np.array([np.random.uniform(w_*2/6, w_*7/8), np.random.uniform(h_/8, h_*7/8)])
cue_b = Ball(p_cb, ax, vel=[0,0], color=(1,1,1))
eight = Ball(p_8b, ax, vel=[0,0], color=(.1,.1,.1))
cue_b.vel = estimate_cue_vel(cue_b, eight, p_holes)

# Time period
t_f = 5
t = np.linspace(0, t_f, num=t_f*200)
dt = np.diff(t)
dt = np.append(dt, [dt[-1]])

def update(dt_i):
    if cue_b.get_visible() and eight.get_visible():
        collision(cue_b, eight)
    cue_b.update(dt_i)
    eight.update(dt_i)
    for hi, phi in zip(holes, p_holes):
        if np.linalg.norm(cue_b.get_xy() - phi) < 0.5*cue_b.radius + hi.radius:
            hi.set_color((1,0,0))
            cue_b.set_visible(False)
        if np.linalg.norm(eight.get_xy() - phi) < 0.5*eight.radius + hi.radius:
            hi.set_color((0,1,0))
            eight.set_visible(False)

ani = matplotlib.animation.FuncAnimation(fig, update, frames=dt, interval=1000*np.mean(dt), repeat=False)
fig.tight_layout()
plt.show()
